# LASSO Deconvolution v3 - Mass Channels

**Approach:** Use Prosit predictions as templates (design matrix X) to estimate mixing coefficients β for chimeric spectra.

**Method:**
- **Channels:** Unique m/z values from all Prosit predictions (mass channels)
- **Matching:** 20 ppm tolerance for observed ↔ predicted
- **Model:** LASSO (L1 regularization) with non-negativity constraint
- **Validation:** Prosit quality check using existing prosit_cosine column

**Hypothesis:**
1. β correlates with FragShare (r > 0.6)
2. β weakly correlates with MS1 (r < 0.3)
3. β outperforms Hyperscore for ranking

## 1. Setup & Configuration

In [1]:
import os
import sys
import json
import pickle
import warnings
from pathlib import Path
from datetime import datetime
from multiprocessing import Pool, cpu_count

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from scipy.optimize import nnls
from scipy.stats import spearmanr

warnings.filterwarnings('ignore')

# Paths
PROJECT_DIR = Path('/data/antwerpen/211/vsc21150/Exploring-Fragmentation-Competion-in-Proteomics-Data-to-Decode-Chimeric-Spectra/v.3.0.0')
DATA_DIR = PROJECT_DIR / 'processed_data'
CACHE_DIR = PROJECT_DIR / 'cache'
PLOT_DIR = PROJECT_DIR / 'plots'
OUTPUT_DIR = PROJECT_DIR / 'analysis' / '07_lasso_deconvolution'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
(OUTPUT_DIR / 'figures').mkdir(exist_ok=True)

# Config
CONFIG = {
    'TOL_PPM': 20,
    'LAMBDA_GRID': np.logspace(-6, 0, 30),
    'N_FOLDS': 5,
    'TRAIN_RATIO': 0.7,
    'RANDOM_SEED': 42,
    'MIN_CHANNELS': 5,
    'MIN_NONZERO': 3,
    'MIN_PSMS': 2,
}

N_CORES = min(64, cpu_count())
np.random.seed(CONFIG['RANDOM_SEED'])

plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.size'] = 11
sns.set_style('whitegrid')

print("="*70)
print("LASSO DECONVOLUTION v3 - MASS CHANNELS")
print("="*70)
print(f"Output: {OUTPUT_DIR}")
print(f"Cores:  {N_CORES}")

mkdir -p failed for path /user/antwerpen/211/vsc21150/.cache/matplotlib: [Errno 122] Disk quota exceeded: '/user/antwerpen/211/vsc21150/.cache/matplotlib'
Matplotlib created a temporary cache directory at /tmp/matplotlib-9el98e3n because there was an issue with the default path (/user/antwerpen/211/vsc21150/.cache/matplotlib); it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Fontconfig error: No writable cache directories
Fontconfig error: No writable cache directories
Fontconfig error: No writable cache directories
Fontconfig error: No writable cache directories
Fontconfig error: No writable cache directories
Fontconfig error: No writable cache directories
Fontconfig error: No writable cache directories


LASSO DECONVOLUTION v3 - MASS CHANNELS
Output: /data/antwerpen/211/vsc21150/Exploring-Fragmentation-Competion-in-Proteomics-Data-to-Decode-Chimeric-Spectra/v.3.0.0/analysis/07_lasso_deconvolution
Cores:  64


/data/antwerpen/211/vsc21150/Exploring-Fragmentation-Competion-in-Proteomics-Data-to-Decode-Chimeric-Spectra/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Install scikit-learn

In [3]:
import subprocess

INSTALL_PATH = "/data/antwerpen/211/vsc21150/python_packages"
os.makedirs(INSTALL_PATH, exist_ok=True)

subprocess.run([
    sys.executable, '-m', 'pip', 'install', 'scikit-learn',
    f'--target={INSTALL_PATH}', '-q'
], check=False, capture_output=True)

sys.path.insert(0, INSTALL_PATH)
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import Lasso

print("✓ scikit-learn ready")

✓ scikit-learn ready


## 3. Utility Functions

In [4]:
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    """Cosine similarity between two vectors."""
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    if norm_a == 0 or norm_b == 0:
        return 0.0
    return float(np.dot(a, b) / (norm_a * norm_b))


def find_match_idx(query_mz: float, ref_mz: np.ndarray, tol_ppm: float) -> int:
    """Find index of closest match within ppm tolerance. Returns -1 if none."""
    if len(ref_mz) == 0:
        return -1
    tol = query_mz * tol_ppm / 1e6
    diffs = np.abs(ref_mz - query_mz)
    idx = np.argmin(diffs)
    return idx if diffs[idx] <= tol else -1


def parse_scan_from_spectrum_key(spec_key: str) -> tuple:
    """Extract (mzml_name, scan) from spectrum_key."""
    parts = spec_key.split('::')
    if len(parts) != 2:
        return None, -1
    
    mzml_name = parts[0]
    
    # Add .mzML extension if missing
    if not mzml_name.endswith('.mzML'):
        mzml_name = mzml_name + '.mzML'
    
    try:
        scan = int(parts[1])
        return mzml_name, scan
    except ValueError:
        return mzml_name, -1


print("✓ Utility functions defined")

✓ Utility functions defined


## 4. Load Data

In [5]:
print("Loading data...")

# PSM data
df = pd.read_csv(CACHE_DIR / 'psm_annotated_final.csv')
print(f"  PSMs: {len(df):,}")

# Prosit predictions
with open(CACHE_DIR / 'prosit_cache.pkl', 'rb') as f:
    prosit_cache = pickle.load(f)
print(f"  Prosit cache: {len(prosit_cache):,}")

# MS2 spectra
with open(CACHE_DIR / 'spectra_dict.pkl', 'rb') as f:
    spectra_dict = pickle.load(f)
print(f"  Spectra: {len(spectra_dict):,}")

# Classify
psm_counts = df.groupby('spectrum_key').size()
singleton_spectra = set(psm_counts[psm_counts == 1].index)
chimeric_spectra = set(psm_counts[psm_counts >= CONFIG['MIN_PSMS']].index)

print(f"  Singleton: {len(singleton_spectra):,}")
print(f"  Chimeric:  {len(chimeric_spectra):,}")
print("✓ Data loaded")

Loading data...
  PSMs: 1,132,669
  Prosit cache: 27,907
  Spectra: 955,800
  Singleton: 296,582
  Chimeric:  314,795
✓ Data loaded


## 5. Prosit Quality Check (SIMPLIFIED)

In [10]:
print("="*70)
print("PROSIT QUALITY CHECK")
print("="*70)

if 'prosit_cosine' in df.columns:
    # Use existing data from 05_prosit.ipynb
    df_singleton = df[df['spectrum_key'].isin(singleton_spectra)].copy()
    cos_vals = df_singleton['prosit_cosine'].dropna()
    
    if len(cos_vals) > 0:
        PROSIT_QUALITY = cos_vals.mean()
        print(f"Prosit quality (from existing data):")
        print(f"  Evaluated: {len(cos_vals):,} singletons")
        print(f"  Mean:   {PROSIT_QUALITY:.4f} ± {cos_vals.std():.4f}")
        print(f"  Median: {cos_vals.median():.4f}")
        print(f"  >0.5:   {100*(cos_vals > 0.5).mean():.1f}%")
        print(f"  >0.7:   {100*(cos_vals > 0.7).mean():.1f}%")
        
        # GATE
        if PROSIT_QUALITY < 0.6:
            print(f"\n⚠️  WARNING: Prosit quality = {PROSIT_QUALITY:.3f} < 0.6!")
            print("    LASSO deconvolution may be unreliable.")
        elif PROSIT_QUALITY < 0.75:
            print(f"\n⚠️  NOTE: Prosit quality = {PROSIT_QUALITY:.3f} is moderate.")
        else:
            print(f"\n✓ Prosit quality = {PROSIT_QUALITY:.3f} is good!")
    else:
        print("⚠️  No singleton Prosit data, skipping quality gate")
        PROSIT_QUALITY = np.nan
else:
    print("⚠️  prosit_cosine not found! Run 05_prosit.ipynb first.")
    PROSIT_QUALITY = np.nan

PROSIT QUALITY CHECK
Prosit quality (from existing data):
  Evaluated: 296,582 singletons
  Mean:   0.7436 ± 0.2423
  Median: 0.8359
  >0.5:   81.5%
  >0.7:   66.2%

⚠️  NOTE: Prosit quality = 0.744 is moderate.


In [11]:
print("="*70)
print("PROSIT QUALITY CHECK")
print("="*70)

# Check Prosit coverage on CHIMERIC spectra (what matters for LASSO)
df_chimeric_check = df[df['spectrum_key'].isin(chimeric_spectra)].copy()
total_chimeric_psms = len(df_chimeric_check)

# Count PSMs with Prosit predictions
psms_with_prosit = 0
for _, row in df_chimeric_check.iterrows():
    key = (row['Peptide'], int(row['Charge']))
    if key in prosit_cache:
        psms_with_prosit += 1

prosit_coverage = psms_with_prosit / total_chimeric_psms if total_chimeric_psms > 0 else 0

print(f"Prosit coverage on CHIMERIC spectra:")
print(f"  Total PSMs:     {total_chimeric_psms:,}")
print(f"  With Prosit:    {psms_with_prosit:,}")
print(f"  Coverage:       {100*prosit_coverage:.1f}%")

# Check quality if prosit_cosine exists
if 'prosit_cosine' in df.columns:
    cos_vals_chimeric = df_chimeric_check['prosit_cosine'].dropna()
    if len(cos_vals_chimeric) > 0:
        PROSIT_QUALITY = cos_vals_chimeric.mean()
        print(f"\nProsit quality on chimeric PSMs:")
        print(f"  Evaluated: {len(cos_vals_chimeric):,}")
        print(f"  Mean:   {PROSIT_QUALITY:.4f} ± {cos_vals_chimeric.std():.4f}")
        print(f"  Median: {cos_vals_chimeric.median():.4f}")
        print(f"  >0.5:   {100*(cos_vals_chimeric > 0.5).mean():.1f}%")
        print(f"  >0.7:   {100*(cos_vals_chimeric > 0.7).mean():.1f}%")
        
        # GATE
        if PROSIT_QUALITY < 0.6:
            print(f"\n⚠️  WARNING: Prosit quality = {PROSIT_QUALITY:.3f} < 0.6!")
            print("    LASSO deconvolution may be unreliable.")
        elif PROSIT_QUALITY < 0.75:
            print(f"\n⚠️  NOTE: Prosit quality = {PROSIT_QUALITY:.3f} is moderate.")
        else:
            print(f"\n✓ Prosit quality = {PROSIT_QUALITY:.3f} is good!")
    else:
        print("\n⚠️  No Prosit cosine data available")
        PROSIT_QUALITY = np.nan
else:
    print("\n⚠️  prosit_cosine column not found")
    PROSIT_QUALITY = np.nan

# Coverage gate
if prosit_coverage < 0.8:
    print(f"\n⚠️  WARNING: Prosit coverage = {prosit_coverage:.1%} < 80%!")
    print("    Many chimeric PSMs will be skipped.")
else:
    print(f"\n✓ Prosit coverage = {prosit_coverage:.1%} is good!")

PROSIT QUALITY CHECK
Prosit coverage on CHIMERIC spectra:
  Total PSMs:     836,087
  With Prosit:    835,809
  Coverage:       100.0%

Prosit quality on chimeric PSMs:
  Evaluated: 836,087
  Mean:   0.6286 ± 0.2515
  Median: 0.6542
  >0.5:   67.3%
  >0.7:   44.7%

⚠️  NOTE: Prosit quality = 0.629 is moderate.

✓ Prosit coverage = 100.0% is good!


## 6. Design Matrix Construction

### Matrix Building (SLURM Job)

In [25]:
import subprocess
import pickle
from pathlib import Path

BASE_DIR = Path('/data/antwerpen/211/vsc21150/Exploring-Fragmentation-Competion-in-Proteomics-Data-to-Decode-Chimeric-Spectra')
CACHE_DIR = BASE_DIR / 'cache'
MATRIX_CACHE = CACHE_DIR / 'spectra_data_chimerys.pkl'
PYTHON_SCRIPT = CACHE_DIR / 'matrix_build_chimerys.py'
SLURM_SCRIPT = CACHE_DIR / 'run_matrix_chimerys.slurm'
SHARED_DATA = CACHE_DIR / 'matrix_job_data.pkl'

# Cancel any running jobs
subprocess.run(['scancel', '-n', 'matrix_chimerys'], capture_output=True)

# Prepare shared data if not exists
if not SHARED_DATA.exists():
    TARGET_SIZE = 100000
    df_chimeric = df[df['spectrum_key'].isin(chimeric_spectra)].copy()
    psm_counts = df_chimeric.groupby('spectrum_key').size()
    
    strata = {n: psm_counts[psm_counts == n].index.tolist() for n in range(2, 20)}
    selected = []
    for n in range(6, 21):
        if strata.get(n): selected.extend(strata[n])
    
    remaining = TARGET_SIZE - len(selected)
    weights = {2: 1, 3: 2, 4: 3, 5: 4}
    total_w = sum(weights[n] * len(strata.get(n, [])) for n in weights)
    
    for n in [2, 3, 4, 5]:
        if not strata.get(n): continue
        alloc = min(int(remaining * weights[n] * len(strata[n]) / total_w), len(strata[n]))
        np.random.seed(CONFIG['RANDOM_SEED'] + n)
        selected.extend(np.random.choice(strata[n], alloc, replace=False).tolist())
    
    df_chimeric = df_chimeric[df_chimeric['spectrum_key'].isin(selected)]
    psm_groups = {k: g.to_dict('records') for k, g in df_chimeric.groupby('spectrum_key')}
    
    with open(SHARED_DATA, 'wb') as f:
        pickle.dump({
            'unique_chimeric': selected, 'psm_groups': psm_groups,
            'spectra_dict': spectra_dict, 'prosit_cache': prosit_cache, 'CONFIG': CONFIG
        }, f)
    print(f"Prepared {len(selected):,} spectra | {SHARED_DATA.stat().st_size/1e6:.1f} MB")
else:
    print(f"Shared data exists: {SHARED_DATA.stat().st_size/1e6:.1f} MB")

# Python script with CHIMERYS-style matrix construction
python_code = f'''#!/usr/bin/env python
import os
os.environ['NUMBA_CACHE_DIR'] = '/tmp'
import numpy as np
import pickle
from multiprocessing import Pool

with open("{SHARED_DATA}", 'rb') as f:
    data = pickle.load(f)

unique_chimeric = data['unique_chimeric']
psm_groups = data['psm_groups']
spectra_dict = data['spectra_dict']
prosit_cache = data['prosit_cache']
CONFIG = data['CONFIG']
del data
print(f"Loaded {{len(unique_chimeric):,}} spectra", flush=True)

def parse_scan(spec_key):
    if '::' in spec_key:
        parts = spec_key.rsplit('::', 1)
        mzml = parts[0] + '.mzML' if not parts[0].endswith('.mzML') else parts[0]
        return mzml, int(parts[1])
    return None, -1

def build_matrix_chimerys(obs_mz, obs_int, prosit_list, tol_ppm, min_channels=3):
    """CHIMERYS-style: only Prosit channels, no unmatched observed peaks."""
    n_pep = len(prosit_list)
    if n_pep == 0:
        return None, None, None
    
    # Collect all Prosit peaks with per-peptide normalization
    p_mz, p_int, p_idx = [], [], []
    for j, p in enumerate(prosit_list):
        mz_arr = np.array(p['mz'])
        int_arr = np.array(p['int'])
        if int_arr.sum() > 0:
            int_arr = int_arr / int_arr.sum()
        p_mz.extend(mz_arr)
        p_int.extend(int_arr)
        p_idx.extend([j] * len(mz_arr))
    
    p_mz = np.array(p_mz)
    p_int = np.array(p_int)
    p_idx = np.array(p_idx)
    
    if len(p_mz) == 0:
        return None, None, None
    
    # Sort by m/z
    sort_idx = np.argsort(p_mz)
    p_mz = p_mz[sort_idx]
    p_int = p_int[sort_idx]
    p_idx = p_idx[sort_idx]
    
    # Group into channels (merge peaks within tolerance)
    channels = []
    channel_contrib = []
    
    curr_mz = p_mz[0]
    curr_contrib = [(p_idx[0], p_int[0])]
    
    for i in range(1, len(p_mz)):
        tol = curr_mz * tol_ppm / 1e6
        if abs(p_mz[i] - curr_mz) <= tol:
            curr_contrib.append((p_idx[i], p_int[i]))
            curr_mz = (curr_mz + p_mz[i]) / 2
        else:
            channels.append(curr_mz)
            channel_contrib.append(curr_contrib)
            curr_mz = p_mz[i]
            curr_contrib = [(p_idx[i], p_int[i])]
    
    channels.append(curr_mz)
    channel_contrib.append(curr_contrib)
    
    ch_mz = np.array(channels)
    n_ch = len(ch_mz)
    
    if n_ch < min_channels:
        return None, None, None
    
    # Build X matrix
    X = np.zeros((n_ch, n_pep))
    for k, contrib in enumerate(channel_contrib):
        for pep_idx, intensity in contrib:
            X[k, pep_idx] += intensity
    
    # Build y: match observed to channels
    y = np.zeros(n_ch)
    for k in range(n_ch):
        tol = ch_mz[k] * tol_ppm / 1e6
        diffs = np.abs(obs_mz - ch_mz[k])
        if len(diffs) > 0:
            min_idx = np.argmin(diffs)
            if diffs[min_idx] <= tol:
                y[k] = obs_int[min_idx]
    
    # Normalize y
    if y.sum() > 0:
        y = y / y.sum()
    else:
        return None, None, None
    
    return y, X, ch_mz

def process(spec_key):
    mzml, scan = parse_scan(spec_key)
    if scan < 0 or (mzml, scan) not in spectra_dict:
        return None
    
    obs = spectra_dict[(mzml, scan)]
    obs_mz = np.array(obs['mz'])
    obs_int = np.array(obs['intensity'])
    psm_rec = psm_groups.get(spec_key, [])
    
    if len(obs_mz) == 0 or len(psm_rec) == 0:
        return None
    
    prosit_list, psm_info = [], []
    for r in psm_rec:
        key = (r['Peptide'], int(r['Charge']))
        if key in prosit_cache:
            prosit_list.append({{'mz': prosit_cache[key]['mz'], 'int': prosit_cache[key]['intensity']}})
            psm_info.append({{
                'peptide': r['Peptide'], 
                'charge': int(r['Charge']),
                'hyperscore': r['Hyperscore'], 
                'by_int_frac': r.get('by_int_frac', np.nan),
                'ms1_intensity': r.get('ms1_combined', np.nan), 
                'prosit_cosine': r.get('prosit_cosine', np.nan)
            }})
    
    if len(prosit_list) < CONFIG['MIN_PSMS']:
        return None
    
    y, X, ch_mz = build_matrix_chimerys(obs_mz, obs_int, prosit_list, CONFIG['TOL_PPM'])
    if y is None:
        return None
    
    return {{
        'spectrum_key': spec_key, 
        'y': y, 
        'X': X, 
        'channel_mz': ch_mz,
        'psm_info': psm_info, 
        'n_psm': len(psm_info), 
        'n_channels': len(y)
    }}

if __name__ == '__main__':
    print(f"Processing with 32 workers", flush=True)
    
    with Pool(32) as pool:
        results = []
        for i, res in enumerate(pool.imap_unordered(process, unique_chimeric, chunksize=200)):
            if res:
                results.append(res)
            if (i + 1) % 20000 == 0:
                print(f"{{i+1:,}}/{{len(unique_chimeric):,}} | Built: {{len(results):,}}", flush=True)
    
    print(f"Total: {{len(results):,}} matrices", flush=True)
    
    with open("{MATRIX_CACHE}", 'wb') as f:
        pickle.dump(results, f)
    print("Saved", flush=True)
'''

with open(PYTHON_SCRIPT, 'w') as f:
    f.write(python_code)

# SLURM script
slurm_code = f'''#!/bin/bash
#SBATCH --job-name=matrix_chimerys
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=32
#SBATCH --time=02:00:00
#SBATCH --mem=50G
#SBATCH --partition=zen2
#SBATCH --output={CACHE_DIR}/matrix_chimerys_%j.out
#SBATCH --error={CACHE_DIR}/matrix_chimerys_%j.err

echo "Started $(date)"
python {PYTHON_SCRIPT}
echo "Finished $(date)"
'''

with open(SLURM_SCRIPT, 'w') as f:
    f.write(slurm_code)

# Remove old cache and submit
if MATRIX_CACHE.exists():
    MATRIX_CACHE.unlink()

result = subprocess.run(['sbatch', str(SLURM_SCRIPT)], capture_output=True, text=True)
if result.returncode == 0:
    print(f"Job {result.stdout.strip().split()[-1]} submitted")
else:
    print(f"Failed: {result.stderr}")

Shared data exists: 4371.9 MB
Job 3027399 submitted


In [15]:
import subprocess
from pathlib import Path

CACHE_DIR = Path('/data/antwerpen/211/vsc21150/Exploring-Fragmentation-Competion-in-Proteomics-Data-to-Decode-Chimeric-Spectra/cache')
MATRIX_CACHE = CACHE_DIR / 'spectra_data_chimerys.pkl'

result = subprocess.run(['squeue', '-u', 'vsc21150', '-n', 'matrix_chimerys'], capture_output=True, text=True)
print(result.stdout if result.stdout.strip().count('\n') > 0 else "No job running")

logs = sorted(CACHE_DIR.glob('matrix_chimerys_*.out'), key=lambda x: x.stat().st_mtime)
if logs:
    print(f"\n=== {logs[-1].name} ===")
    subprocess.run(['tail', '-15', str(logs[-1])])

if MATRIX_CACHE.exists():
    print(f"\n✓ Results ready: {MATRIX_CACHE.stat().st_size/1e6:.1f} MB")

No job running

=== matrix_chimerys_3027399.out ===
Started Sun Jan 25 02:59:31 PM CET 2026
Loaded 99,999 spectra
Processing with 32 workers
20,000/99,999 | Built: 19,982
40,000/99,999 | Built: 39,972
60,000/99,999 | Built: 59,972
80,000/99,999 | Built: 79,971
Total: 99,970 matrices
Saved
Finished Sun Jan 25 02:59:55 PM CET 2026

✓ Results ready: 296.8 MB


In [6]:
import pickle
import numpy as np
from pathlib import Path

CACHE_DIR = Path('/data/antwerpen/211/vsc21150/Exploring-Fragmentation-Competion-in-Proteomics-Data-to-Decode-Chimeric-Spectra/cache')
MATRIX_CACHE = CACHE_DIR / 'spectra_data_chimerys.pkl'

with open(MATRIX_CACHE, 'rb') as f:
    spectra_data = pickle.load(f)

print(f"Loaded {len(spectra_data):,} matrices")
print(f"PSMs: {np.unique([sd['n_psm'] for sd in spectra_data], return_counts=True)}")



Loaded 99,970 matrices
PSMs: (array([2, 3, 4, 5]), array([33159, 34630, 22254,  9927]))


In [29]:
# Quick validation
n_ch = [sd['n_channels'] for sd in spectra_data]
print(f"Channels: min={min(n_ch)}, max={max(n_ch)}, mean={np.mean(n_ch):.1f}")

# Test NNLS on sample
from scipy.optimize import nnls

cosines = []
for sd in spectra_data[:1000]:
    y, X = sd['y'], sd['X']
    beta, _ = nnls(X, y)
    y_pred = X @ beta
    cos = np.dot(y, y_pred) / (np.linalg.norm(y) * np.linalg.norm(y_pred) + 1e-10)
    cosines.append(cos)

print(f"\nNNLS cosine (n=1000): {np.mean(cosines):.3f} ± {np.std(cosines):.3f}")
print(f"  >0.5: {100*np.mean(np.array(cosines) > 0.5):.1f}%")
print(f"  >0.7: {100*np.mean(np.array(cosines) > 0.7):.1f}%")

Channels: min=14, max=220, mean=57.6

NNLS cosine (n=1000): 0.773 ± 0.193
  >0.5: 88.6%
  >0.7: 69.5%


## LASSO + ms1

In [7]:
# ============================================================
# LASSO ANALYSIS: DATASET PREPARATION
# ============================================================
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from scipy.optimize import nnls
from multiprocessing import Pool, cpu_count
import time

print("="*70)
print("LASSO ANALYSIS: THREE MODELS")
print("="*70)
print("Model 1: LASSO (2+ PSM)")
print("Model 2: LASSO (3+ PSM)")
print("Model 3: LASSO+MS1 (3+ PSM)")
print("="*70)

# Build PSM dataframe from spectra_data
results_list = []
for sd in spectra_data:
    for j, psm in enumerate(sd['psm_info']):
        results_list.append({
            'spectrum_key': sd['spectrum_key'],
            'peptide': psm['peptide'],
            'charge': psm['charge'],
            'hyperscore': psm['hyperscore'],
            'by_int_frac': psm.get('by_int_frac', np.nan),
            'prosit_cosine': psm.get('prosit_cosine', np.nan),
            'ms1_intensity': psm.get('ms1_intensity', 0),
            'n_psm': sd['n_psm']
        })

df_results_all = pd.DataFrame(results_list)

print(f"\nComplete dataset (2+ PSM):")
print(f"  Total matrices: {len(spectra_data):,}")
print(f"  Total PSMs:     {len(df_results_all):,}")
print(f"  Total spectra:  {df_results_all['spectrum_key'].nunique():,}")

# PSM distribution
psm_dist = df_results_all.groupby('spectrum_key')['n_psm'].first().value_counts().sort_index()
print(f"\nPSM distribution:")
for n, count in psm_dist.items():
    print(f"  {n} PSM: {count:,} spectra")

LASSO ANALYSIS: THREE MODELS
Model 1: LASSO (2+ PSM)
Model 2: LASSO (3+ PSM)
Model 3: LASSO+MS1 (3+ PSM)

Complete dataset (2+ PSM):
  Total matrices: 99,970
  Total PSMs:     308,859
  Total spectra:  99,970

PSM distribution:
  2 PSM: 33,159 spectra
  3 PSM: 34,630 spectra
  4 PSM: 22,254 spectra
  5 PSM: 9,927 spectra


In [8]:
# ============================================================
# MERGE WITH MS1 DATA FROM psm_clean.csv
# ============================================================

print(f"\n{'='*70}")
print("MS1 DATA INTEGRATION")
print(f"{'='*70}")

print(f"\nLoading MS1 data from psm_clean.csv...")
df_clean = pd.read_csv(DATA_DIR / 'psm_clean.csv', 
                       usecols=['spectrum_key', 'Peptide', 'Intensity'])
df_clean = df_clean.rename(columns={'Peptide': 'peptide', 'Intensity': 'ms1_intensity'})
print(f"  Loaded: {len(df_clean):,} rows")

# Merge
df_results_all = df_results_all.drop(columns=['ms1_intensity'], errors='ignore')
df_results_all = df_results_all.merge(
    df_clean[['spectrum_key', 'peptide', 'ms1_intensity']],
    on=['spectrum_key', 'peptide'],
    how='left'
)

print(f"  After merge: {len(df_results_all):,} PSMs")
print(f"  PSMs with MS1 > 0: {(df_results_all['ms1_intensity'] > 0).sum():,}")

# Filter to spectra with complete MS1 (for LASSO+MS1 model only)
def has_complete_ms1(group):
    """Check if all peptides in spectrum have MS1 > 0"""
    return (group['ms1_intensity'] > 0).all()

print(f"\nFiltering to spectra with complete MS1 (for LASSO+MS1 model)...")
spectra_with_ms1 = df_results_all.groupby('spectrum_key').filter(has_complete_ms1)
unique_spectra_ms1 = spectra_with_ms1['spectrum_key'].unique()
unique_spectra_ms1_set = set(unique_spectra_ms1)

print(f"  Spectra with complete MS1: {len(unique_spectra_ms1):,}")
print(f"  PSMs with complete MS1: {len(spectra_with_ms1):,}")

# PSM distribution for MS1-complete spectra
psm_dist_ms1 = spectra_with_ms1.groupby('spectrum_key')['n_psm'].first().value_counts().sort_index()
print(f"\n  PSM distribution (MS1 complete):")
for n, count in psm_dist_ms1.items():
    print(f"    {n} PSM: {count:,} spectra ({100*count/len(unique_spectra_ms1):.1f}%)")

# Build MS1 lookup dictionary (for LASSO+MS1 model)
print(f"\nBuilding MS1 lookup dictionary...")
ms1_dict = {}
for _, row in spectra_with_ms1.iterrows():
    key = (row['spectrum_key'], row['peptide'])
    ms1_dict[key] = row['ms1_intensity']

print(f"  MS1 entries: {len(ms1_dict):,}")

print(f"\n{'='*70}")
print("DATASET SUMMARY")
print(f"{'='*70}")

print(f"\nOriginal dataset (2+ PSM):")
print(f"  Total spectra:    {len(spectra_data):,}")
print(f"  Total PSMs:       {len(df_results_all):,}")

print(f"\nFiltered dataset (MS1 complete, for Model 3):")
print(f"  Total spectra:    {len(unique_spectra_ms1):,}")
print(f"  Total PSMs:       {len(spectra_with_ms1):,}")
print(f"  Coverage:         {100*len(unique_spectra_ms1)/len(spectra_data):.1f}%")

print(f"\n{'='*70}")
print("✓ MS1 data ready")
print(f"{'='*70}")


MS1 DATA INTEGRATION

Loading MS1 data from psm_clean.csv...
  Loaded: 1,276,641 rows
  After merge: 308,891 PSMs
  PSMs with MS1 > 0: 280,214

Filtering to spectra with complete MS1 (for LASSO+MS1 model)...
  Spectra with complete MS1: 79,396
  PSMs with complete MS1: 240,907

  PSM distribution (MS1 complete):
    2 PSM: 28,209 spectra (35.5%)
    3 PSM: 27,449 spectra (34.6%)
    4 PSM: 16,572 spectra (20.9%)
    5 PSM: 7,166 spectra (9.0%)

Building MS1 lookup dictionary...
  MS1 entries: 240,871

DATASET SUMMARY

Original dataset (2+ PSM):
  Total spectra:    99,970
  Total PSMs:       308,891

Filtered dataset (MS1 complete, for Model 3):
  Total spectra:    79,396
  Total PSMs:       240,907
  Coverage:         79.4%

✓ MS1 data ready


In [9]:
# ============================================================
# PREPARE DATASETS FOR THREE MODELS
# ============================================================

def add_ms1_to_matrix(sd, ms1_dict):
    """
    Add MS1 as additional features to design matrix.
    Strategy: Add MS1 as "soft prior" channels
    Each peptide gets a diagonal row with its normalized MS1
    """
    spec_key = sd['spectrum_key']
    X = sd['X']
    n_channels, n_pep = X.shape
    
    # Extract MS1 for each peptide
    ms1_values = []
    for psm in sd['psm_info']:
        peptide = psm['peptide']
        ms1 = ms1_dict.get((spec_key, peptide), 0)
        ms1_values.append(ms1)
    
    ms1_values = np.array(ms1_values)
    
    # Skip if any MS1 missing or zero
    if (ms1_values <= 0).any():
        return None
    
    # Normalize MS1 to sum = 1
    ms1_norm = ms1_values / ms1_values.sum()
    
    # Augment design matrix
    ms1_channels = np.diag(ms1_norm)
    X_augmented = np.vstack([X, ms1_channels])
    y_augmented = np.concatenate([sd['y'], np.zeros(n_pep)])
    
    return {
        'spectrum_key': spec_key,
        'y': sd['y'],
        'y_augmented': y_augmented,
        'X_base': X,
        'X_augmented': X_augmented,
        'ms1_values': ms1_values,
        'ms1_norm': ms1_norm,
        'psm_info': sd['psm_info'],
        'n_psm': sd['n_psm'],
        'n_channels': n_channels
    }

print(f"\n{'='*70}")
print("PREPARING DATASETS FOR THREE MODELS")
print(f"{'='*70}")

# Model 1: LASSO (2+ PSM) - uses all spectra as-is
spectra_2plus = spectra_data.copy()
print(f"\nModel 1: LASSO (2+ PSM)")
print(f"  Spectra: {len(spectra_2plus):,}")

# Model 2: LASSO (3+ PSM) - filter to 3+ PSM only
spectra_3plus = [sd for sd in spectra_data if sd['n_psm'] >= 3]
print(f"\nModel 2: LASSO (3+ PSM)")
print(f"  Spectra: {len(spectra_3plus):,}")

# Model 3: LASSO+MS1 (3+ PSM) - filter to 3+ PSM with complete MS1, then augment
print(f"\nModel 3: LASSO+MS1 (3+ PSM)")
print(f"  Filtering to 3+ PSM with complete MS1...")
spectra_3plus_ms1_temp = [sd for sd in spectra_data 
                          if sd['n_psm'] >= 3 and sd['spectrum_key'] in unique_spectra_ms1_set]

print(f"  Adding MS1 to design matrices...")
spectra_3plus_ms1 = []
skipped = 0

for sd in spectra_3plus_ms1_temp:
    result = add_ms1_to_matrix(sd, ms1_dict)
    if result is not None:
        spectra_3plus_ms1.append(result)
    else:
        skipped += 1

print(f"  Augmented matrices: {len(spectra_3plus_ms1):,}")
print(f"  Skipped: {skipped:,}")

# Train/val splits for all three models
print(f"\n{'='*70}")
print("TRAIN/VALIDATION SPLITS")
print(f"{'='*70}")

# Model 1: LASSO (2+ PSM)
indices_2plus = list(range(len(spectra_2plus)))
train_idx_2plus, val_idx_2plus = train_test_split(indices_2plus, test_size=0.3, random_state=42)
train_2plus = [spectra_2plus[i] for i in train_idx_2plus]
val_2plus = [spectra_2plus[i] for i in val_idx_2plus]

print(f"\nModel 1: LASSO (2+ PSM)")
print(f"  Train: {len(train_2plus):,}")
print(f"  Val:   {len(val_2plus):,}")

# Model 2: LASSO (3+ PSM)
indices_3plus = list(range(len(spectra_3plus)))
train_idx_3plus, val_idx_3plus = train_test_split(indices_3plus, test_size=0.3, random_state=42)
train_3plus = [spectra_3plus[i] for i in train_idx_3plus]
val_3plus = [spectra_3plus[i] for i in val_idx_3plus]

print(f"\nModel 2: LASSO (3+ PSM)")
print(f"  Train: {len(train_3plus):,}")
print(f"  Val:   {len(val_3plus):,}")

# Model 3: LASSO+MS1 (3+ PSM)
indices_3plus_ms1 = list(range(len(spectra_3plus_ms1)))
train_idx_3plus_ms1, val_idx_3plus_ms1 = train_test_split(indices_3plus_ms1, test_size=0.3, random_state=42)
train_3plus_ms1 = [spectra_3plus_ms1[i] for i in train_idx_3plus_ms1]
val_3plus_ms1 = [spectra_3plus_ms1[i] for i in val_idx_3plus_ms1]

print(f"\nModel 3: LASSO+MS1 (3+ PSM)")
print(f"  Train: {len(train_3plus_ms1):,}")
print(f"  Val:   {len(val_3plus_ms1):,}")

print(f"\n{'='*70}")
print("✓ All datasets prepared")
print(f"{'='*70}")


PREPARING DATASETS FOR THREE MODELS

Model 1: LASSO (2+ PSM)
  Spectra: 99,970

Model 2: LASSO (3+ PSM)
  Spectra: 66,811

Model 3: LASSO+MS1 (3+ PSM)
  Filtering to 3+ PSM with complete MS1...
  Adding MS1 to design matrices...
  Augmented matrices: 51,187
  Skipped: 0

TRAIN/VALIDATION SPLITS

Model 1: LASSO (2+ PSM)
  Train: 69,979
  Val:   29,991

Model 2: LASSO (3+ PSM)
  Train: 46,767
  Val:   20,044

Model 3: LASSO+MS1 (3+ PSM)
  Train: 35,830
  Val:   15,357

✓ All datasets prepared


In [10]:
# ============================================================
# LAMBDA GRID SEARCH: THREE MODELS
# ============================================================

from collections import defaultdict

def cosine(a, b):
    n = np.linalg.norm(a) * np.linalg.norm(b)
    return np.dot(a, b) / n if n > 1e-10 else 0.0

def evaluate_lasso(args):
    sd, use_ms1, lambda_val, model_name = args
    
    if use_ms1:
        X = sd['X_augmented']
        y = sd['y_augmented']
        X_base = sd['X_base']
        y_base = sd['y']
    else:
        X = sd['X']
        y = sd['y']
        X_base = X
        y_base = y
    
    # Fit LASSO
    if lambda_val == 0:
        beta, _ = nnls(X, y)
    else:
        model = Lasso(alpha=lambda_val, positive=True,
                     fit_intercept=False, max_iter=10000, tol=1e-8)
        model.fit(X, y)
        beta = model.coef_
    
    # Predict on spectral part
    y_pred = X_base @ beta[:sd['n_psm']]
    cos = cosine(y_base, y_pred)
    
    # Sparsity
    k = np.sum(beta[:sd['n_psm']] > 1e-6)
    
    return {
        'spectrum_key': sd['spectrum_key'],
        'model': model_name,
        'lambda': lambda_val,
        'cosine': cos,
        'k': k,
        'n_psm': sd['n_psm']
    }

# Lambda grid
LAMBDA_GRID = np.array([0, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2])

print(f"\n{'='*70}")
print("LAMBDA GRID SEARCH")
print(f"{'='*70}")
print(f"\nGrid: {len(LAMBDA_GRID)} values")

all_results = []

# Model 1: LASSO (2+ PSM)
print(f"\n--- Model 1: LASSO (2+ PSM) ---")
print(f"Training spectra: {len(train_2plus):,}")

for lam in LAMBDA_GRID:
    tasks = [(sd, False, lam, 'LASSO (2+ PSM)') for sd in train_2plus]
    
    t0 = time.time()
    with Pool(cpu_count()) as pool:
        results = pool.map(evaluate_lasso, tasks, chunksize=200)
    t1 = time.time()
    
    all_results.extend(results)
    
    mean_cos = np.mean([r['cosine'] for r in results])
    mean_k = np.mean([r['k'] for r in results])
    lam_str = "NNLS" if lam == 0 else f"{lam:.0e}"
    
    print(f"  λ={lam_str:>8s}: cosine={mean_cos:.4f}, k={mean_k:.1f}, time={t1-t0:.1f}s")

# Model 2: LASSO (3+ PSM)
print(f"\n--- Model 2: LASSO (3+ PSM) ---")
print(f"Training spectra: {len(train_3plus):,}")

for lam in LAMBDA_GRID:
    tasks = [(sd, False, lam, 'LASSO (3+ PSM)') for sd in train_3plus]
    
    t0 = time.time()
    with Pool(cpu_count()) as pool:
        results = pool.map(evaluate_lasso, tasks, chunksize=200)
    t1 = time.time()
    
    all_results.extend(results)
    
    mean_cos = np.mean([r['cosine'] for r in results])
    mean_k = np.mean([r['k'] for r in results])
    lam_str = "NNLS" if lam == 0 else f"{lam:.0e}"
    
    print(f"  λ={lam_str:>8s}: cosine={mean_cos:.4f}, k={mean_k:.1f}, time={t1-t0:.1f}s")

# Model 3: LASSO+MS1 (3+ PSM)
print(f"\n--- Model 3: LASSO+MS1 (3+ PSM) ---")
print(f"Training spectra: {len(train_3plus_ms1):,}")

for lam in LAMBDA_GRID:
    tasks = [(sd, True, lam, 'LASSO+MS1 (3+ PSM)') for sd in train_3plus_ms1]
    
    t0 = time.time()
    with Pool(cpu_count()) as pool:
        results = pool.map(evaluate_lasso, tasks, chunksize=200)
    t1 = time.time()
    
    all_results.extend(results)
    
    mean_cos = np.mean([r['cosine'] for r in results])
    mean_k = np.mean([r['k'] for r in results])
    lam_str = "NNLS" if lam == 0 else f"{lam:.0e}"
    
    print(f"  λ={lam_str:>8s}: cosine={mean_cos:.4f}, k={mean_k:.1f}, time={t1-t0:.1f}s")

df_grid = pd.DataFrame(all_results)
print(f"\n✓ Grid search complete for all three models")


LAMBDA GRID SEARCH

Grid: 8 values

--- Model 1: LASSO (2+ PSM) ---
Training spectra: 69,979
  λ=    NNLS: cosine=0.7942, k=3.1, time=6.2s
  λ=   1e-08: cosine=0.7942, k=3.1, time=7.5s
  λ=   1e-07: cosine=0.7942, k=3.1, time=8.3s
  λ=   1e-06: cosine=0.7942, k=3.1, time=10.2s
  λ=   1e-05: cosine=0.7942, k=3.1, time=8.2s
  λ=   1e-04: cosine=0.7897, k=2.8, time=7.8s
  λ=   1e-03: cosine=0.5197, k=0.9, time=8.5s
  λ=   1e-02: cosine=0.0058, k=0.0, time=8.4s

--- Model 2: LASSO (3+ PSM) ---
Training spectra: 46,767
  λ=    NNLS: cosine=0.8024, k=3.6, time=7.8s
  λ=   1e-08: cosine=0.8024, k=3.6, time=7.6s
  λ=   1e-07: cosine=0.8024, k=3.6, time=8.4s
  λ=   1e-06: cosine=0.8024, k=3.6, time=7.6s
  λ=   1e-05: cosine=0.8023, k=3.6, time=7.9s
  λ=   1e-04: cosine=0.7957, k=3.2, time=8.3s
  λ=   1e-03: cosine=0.4227, k=0.7, time=8.8s
  λ=   1e-02: cosine=0.0001, k=0.0, time=8.8s

--- Model 3: LASSO+MS1 (3+ PSM) ---
Training spectra: 35,830
  λ=    NNLS: cosine=0.7462, k=3.6, time=8.6s
  λ

In [11]:
# ============================================================
# SELECT OPTIMAL LAMBDA
# ============================================================

print(f"\n{'='*70}")
print("OPTIMAL LAMBDA SELECTION")
print(f"{'='*70}")

# Aggregate by model and lambda
summary = df_grid.groupby(['model', 'lambda']).agg({
    'cosine': ['mean', 'std'],
    'k': 'mean'
}).round(4)

summary.columns = ['Cosine_Mean', 'Cosine_Std', 'k_Mean']

print(f"\n{'Model':25s} {'λ':>10s} {'Cosine':>8s} {'Std':>8s} {'k':>6s}")
print("-" * 70)

for (model, lam), row in summary.iterrows():
    lam_str = "NNLS" if lam == 0 else f"{lam:.0e}"
    print(f"{model:25s} {lam_str:>10s} {row['Cosine_Mean']:>8.4f} "
          f"{row['Cosine_Std']:>8.4f} {row['k_Mean']:>6.1f}")

# Find optimal lambda for each model
optimal = {}
for model in ['LASSO (2+ PSM)', 'LASSO (3+ PSM)', 'LASSO+MS1 (3+ PSM)']:
    subset = summary.loc[model]
    valid = subset[subset['k_Mean'] > 0]
    if len(valid) > 0:
        best_lam = valid['Cosine_Mean'].idxmax()
        optimal[model] = best_lam
    else:
        optimal[model] = 0

print(f"\n{'='*70}")
print("OPTIMAL LAMBDAS")
print(f"{'='*70}")

for model, lam in optimal.items():
    lam_str = "NNLS (0)" if lam == 0 else f"{lam:.0e}"
    cos_mean = summary.loc[(model, lam), 'Cosine_Mean']
    k_mean = summary.loc[(model, lam), 'k_Mean']
    print(f"\n{model}:")
    print(f"  λ*:     {lam_str}")
    print(f"  Cosine: {cos_mean:.4f}")
    print(f"  k:      {k_mean:.1f}")

print(f"\n{'='*70}")
print("✓ Optimal lambdas selected")
print(f"{'='*70}")


OPTIMAL LAMBDA SELECTION

Model                              λ   Cosine      Std      k
----------------------------------------------------------------------
LASSO (2+ PSM)                  NNLS   0.7942   0.1657    3.1
LASSO (2+ PSM)                 1e-08   0.7942   0.1657    3.1
LASSO (2+ PSM)                 1e-07   0.7942   0.1657    3.1
LASSO (2+ PSM)                 1e-06   0.7942   0.1657    3.1
LASSO (2+ PSM)                 1e-05   0.7942   0.1657    3.1
LASSO (2+ PSM)                 1e-04   0.7897   0.1658    2.8
LASSO (2+ PSM)                 1e-03   0.5197   0.3869    0.9
LASSO (2+ PSM)                 1e-02   0.0058   0.0735    0.0
LASSO (3+ PSM)                  NNLS   0.8024   0.1507    3.6
LASSO (3+ PSM)                 1e-08   0.8024   0.1507    3.6
LASSO (3+ PSM)                 1e-07   0.8024   0.1507    3.6
LASSO (3+ PSM)                 1e-06   0.8024   0.1507    3.6
LASSO (3+ PSM)                 1e-05   0.8023   0.1507    3.6
LASSO (3+ PSM)                 1e-

In [12]:
# ============================================================
# REGULARIZATION PATH ANALYSIS (VISUAL ONLY - SMALL SAMPLE)
# ============================================================
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import Lasso
from scipy.optimize import nnls
import time

print("="*70)
print("REGULARIZATION PATH ANALYSIS")
print("="*70)

# Small sample for visualization only
SAMPLE_SIZE = 10
LAMBDA_PATH = np.logspace(-8, 0, 30)

print(f"\nSampling {SAMPLE_SIZE} spectra per model (for visualization)")
print(f"Lambda grid: {len(LAMBDA_PATH)} points")

np.random.seed(42)

# Sample from each training set
sample_2plus = np.random.choice(train_2plus, min(SAMPLE_SIZE, len(train_2plus)), replace=False)
sample_3plus = np.random.choice(train_3plus, min(SAMPLE_SIZE, len(train_3plus)), replace=False)
sample_3plus_ms1 = np.random.choice(train_3plus_ms1, min(SAMPLE_SIZE, len(train_3plus_ms1)), replace=False)

# ============================================================
# COMPUTE PATHS
# ============================================================

def compute_path(sd, lambdas, use_ms1):
    """Compute LASSO path for one spectrum"""
    if use_ms1:
        X = sd['X_augmented']
        y = sd['y_augmented']
        X_base = sd['X_base']
        y_base = sd['y']
    else:
        X = sd['X']
        y = sd['y']
        X_base = X
        y_base = y
    
    n_pep = sd['n_psm']
    cosines = []
    k_ratios = []
    
    for lam in lambdas:
        if lam == 0:
            beta, _ = nnls(X, y)
        else:
            model = Lasso(alpha=lam, positive=True, fit_intercept=False, 
                         max_iter=10000, tol=1e-8)
            model.fit(X, y)
            beta = model.coef_
        
        beta_pep = beta[:n_pep]
        y_pred = X_base @ beta_pep
        cos = np.dot(y_base, y_pred) / (np.linalg.norm(y_base) * 
                                         np.linalg.norm(y_pred) + 1e-10)
        cosines.append(cos)
        k_ratios.append(np.sum(beta_pep > 1e-6) / n_pep)
    
    return np.array(cosines), np.array(k_ratios)

# Compute paths for all three models
print("\nComputing paths...")

cosines_2plus_list = []
k_2plus_list = []
for sd in sample_2plus:
    cos, k = compute_path(sd, LAMBDA_PATH, use_ms1=False)
    cosines_2plus_list.append(cos)
    k_2plus_list.append(k)

cosines_3plus_list = []
k_3plus_list = []
for sd in sample_3plus:
    cos, k = compute_path(sd, LAMBDA_PATH, use_ms1=False)
    cosines_3plus_list.append(cos)
    k_3plus_list.append(k)

cosines_3plus_ms1_list = []
k_3plus_ms1_list = []
for sd in sample_3plus_ms1:
    cos, k = compute_path(sd, LAMBDA_PATH, use_ms1=True)
    cosines_3plus_ms1_list.append(cos)
    k_3plus_ms1_list.append(k)

# Aggregate
cos_mean_2plus = np.mean(cosines_2plus_list, axis=0)
cos_mean_3plus = np.mean(cosines_3plus_list, axis=0)
cos_mean_3plus_ms1 = np.mean(cosines_3plus_ms1_list, axis=0)

k_mean_2plus = np.mean(k_2plus_list, axis=0)
k_mean_3plus = np.mean(k_3plus_list, axis=0)
k_mean_3plus_ms1 = np.mean(k_3plus_ms1_list, axis=0)

print(f"✓ Paths computed for {SAMPLE_SIZE} spectra per model")

FIG_DIR = Path('/tmp/lasso_figures')
FIG_DIR.mkdir(exist_ok=True)

REGULARIZATION PATH ANALYSIS

Sampling 10 spectra per model (for visualization)
Lambda grid: 30 points

Computing paths...
✓ Paths computed for 10 spectra per model


In [17]:
# ============================================================
# FIGURE: PERFORMANCE AT OPTIMAL LAMBDA
# ============================================================

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

COLOR_2PLUS = '#E63946'
COLOR_3PLUS = '#F77F00'
COLOR_MS1 = '#457B9D'

# Optimal lambdas
lam_opt_2plus = optimal['LASSO (2+ PSM)']
lam_opt_3plus = optimal['LASSO (3+ PSM)']
lam_opt_ms1 = optimal['LASSO+MS1 (3+ PSM)']

# Extract performance at optimal lambdas
idx_2plus = np.argmin(np.abs(LAMBDA_PATH - (lam_opt_2plus if lam_opt_2plus > 0 else LAMBDA_PATH[0])))
idx_3plus = np.argmin(np.abs(LAMBDA_PATH - (lam_opt_3plus if lam_opt_3plus > 0 else LAMBDA_PATH[0])))
idx_ms1 = np.argmin(np.abs(LAMBDA_PATH - (lam_opt_ms1 if lam_opt_ms1 > 0 else LAMBDA_PATH[0])))

models = ['LASSO\n(2+ PSM)', 'LASSO\n(3+ PSM)', 'LASSO+MS1\n(3+ PSM)']
cosines_at_opt = [cos_mean_2plus[idx_2plus], cos_mean_3plus[idx_3plus], cos_mean_3plus_ms1[idx_ms1]]
colors = [COLOR_2PLUS, COLOR_3PLUS, COLOR_MS1]

bars = ax.bar(models, cosines_at_opt, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

# Add value labels on bars
for bar, val in zip(bars, cosines_at_opt):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.4f}',
            ha='center', va='bottom', fontweight='bold', fontsize=11)

ax.set_ylabel('Mean Cosine', fontweight='bold', fontsize=12)
ax.set_title('Performance at Optimal λ', fontweight='bold', fontsize=13)
ax.grid(alpha=0.3, axis='y')
ax.set_ylim([0, max(cosines_at_opt) * 1.1])

plt.tight_layout()
plt.savefig(FIG_DIR / 'performance_at_optimal_lambda.png', dpi=150, facecolor='white')
plt.show()

print(f"\n✓ Saved: {FIG_DIR / 'performance_at_optimal_lambda.png'}")

<Figure size 960x720 with 1 Axes>


✓ Saved: /tmp/lasso_figures/performance_at_optimal_lambda.png


In [16]:
# ============================================================
# FIGURE: REGULARIZATION PATHS - ONE PLOT PER MODEL
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

COLOR_2PLUS = '#E63946'
COLOR_3PLUS = '#F77F00'
COLOR_MS1 = '#457B9D'

# Optimal lambdas
lam_opt_2plus = optimal['LASSO (2+ PSM)']
lam_opt_3plus = optimal['LASSO (3+ PSM)']
lam_opt_ms1 = optimal['LASSO+MS1 (3+ PSM)']

# ============================================================
# Panel 1: LASSO (2+ PSM)
# ============================================================
ax = axes[0]

# Individual paths
for i, cos_path in enumerate(cosines_2plus_list):
    ax.plot(LAMBDA_PATH, cos_path, linewidth=1.5, alpha=0.5, color=COLOR_2PLUS)

# Mean path
ax.plot(LAMBDA_PATH, cos_mean_2plus, linewidth=3, color='black', 
       label='Mean', zorder=10, linestyle='--')

# Optimal lambda
if lam_opt_2plus > 0:
    ax.axvline(lam_opt_2plus, color=COLOR_2PLUS, linestyle='--', 
              linewidth=2.5, alpha=0.8, label=f'Optimal λ = {lam_opt_2plus:.0e}')
else:
    ax.axvline(LAMBDA_PATH[0], color=COLOR_2PLUS, linestyle='--', 
              linewidth=2.5, alpha=0.8, label='Optimal λ = NNLS')

ax.set_xscale('log')
ax.set_xlabel('λ (Regularization Parameter)', fontweight='bold', fontsize=11)
ax.set_ylabel('Reconstruction Cosine', fontweight='bold', fontsize=11)
ax.set_title(f'LASSO (2+ PSM)\n{SAMPLE_SIZE} sample spectra', 
             fontweight='bold', fontsize=12, color=COLOR_2PLUS)
ax.grid(alpha=0.3)
ax.legend(fontsize=9, loc='lower left')
ax.set_ylim([0, 1.05])

# ============================================================
# Panel 2: LASSO (3+ PSM)
# ============================================================
ax = axes[1]

# Individual paths
for i, cos_path in enumerate(cosines_3plus_list):
    ax.plot(LAMBDA_PATH, cos_path, linewidth=1.5, alpha=0.5, color=COLOR_3PLUS)

# Mean path
ax.plot(LAMBDA_PATH, cos_mean_3plus, linewidth=3, color='black', 
       label='Mean', zorder=10, linestyle='--')

# Optimal lambda
if lam_opt_3plus > 0:
    ax.axvline(lam_opt_3plus, color=COLOR_3PLUS, linestyle='--', 
              linewidth=2.5, alpha=0.8, label=f'Optimal λ = {lam_opt_3plus:.0e}')
else:
    ax.axvline(LAMBDA_PATH[0], color=COLOR_3PLUS, linestyle='--', 
              linewidth=2.5, alpha=0.8, label='Optimal λ = NNLS')

ax.set_xscale('log')
ax.set_xlabel('λ (Regularization Parameter)', fontweight='bold', fontsize=11)
ax.set_ylabel('Reconstruction Cosine', fontweight='bold', fontsize=11)
ax.set_title(f'LASSO (3+ PSM)\n{SAMPLE_SIZE} sample spectra', 
             fontweight='bold', fontsize=12, color=COLOR_3PLUS)
ax.grid(alpha=0.3)
ax.legend(fontsize=9, loc='lower left')
ax.set_ylim([0, 1.05])

# ============================================================
# Panel 3: LASSO+MS1 (3+ PSM)
# ============================================================
ax = axes[2]

# Individual paths
for i, cos_path in enumerate(cosines_3plus_ms1_list):
    ax.plot(LAMBDA_PATH, cos_path, linewidth=1.5, alpha=0.5, color=COLOR_MS1)

# Mean path
ax.plot(LAMBDA_PATH, cos_mean_3plus_ms1, linewidth=3, color='black', 
       label='Mean', zorder=10, linestyle='--')

# Optimal lambda
if lam_opt_ms1 > 0:
    ax.axvline(lam_opt_ms1, color=COLOR_MS1, linestyle='--', 
              linewidth=2.5, alpha=0.8, label=f'Optimal λ = {lam_opt_ms1:.0e}')
else:
    ax.axvline(LAMBDA_PATH[0], color=COLOR_MS1, linestyle='--', 
              linewidth=2.5, alpha=0.8, label='Optimal λ = NNLS')

ax.set_xscale('log')
ax.set_xlabel('λ (Regularization Parameter)', fontweight='bold', fontsize=11)
ax.set_ylabel('Reconstruction Cosine', fontweight='bold', fontsize=11)
ax.set_title(f'LASSO+MS1 (3+ PSM)\n{SAMPLE_SIZE} sample spectra', 
             fontweight='bold', fontsize=12, color=COLOR_MS1)
ax.grid(alpha=0.3)
ax.legend(fontsize=9, loc='lower left')
ax.set_ylim([0, 1.05])

plt.tight_layout()
plt.savefig(FIG_DIR / 'regularization_paths_three_models.png', dpi=150, facecolor='white')
plt.show()

print(f"\n✓ Saved: {FIG_DIR / 'regularization_paths_three_models.png'}")

<Figure size 2160x600 with 3 Axes>


✓ Saved: /tmp/lasso_figures/regularization_paths_three_models.png


In [18]:
# ============================================================
# VALIDATION WITH OPTIMAL LAMBDA
# ============================================================

def validate_full(args):
    sd, use_ms1, lambda_val, model_name = args
    
    if use_ms1:
        X = sd['X_augmented']
        y = sd['y_augmented']
        X_base = sd['X_base']
        y_base = sd['y']
    else:
        X = sd['X']
        y = sd['y']
        X_base = X
        y_base = y
    
    # Fit
    if lambda_val == 0:
        beta, _ = nnls(X, y)
    else:
        model = Lasso(alpha=lambda_val, positive=True,
                     fit_intercept=False, max_iter=10000, tol=1e-8)
        model.fit(X, y)
        beta = model.coef_
    
    # Normalize beta
    beta_peptides = beta[:sd['n_psm']]
    if beta_peptides.sum() > 0:
        beta_norm = beta_peptides / beta_peptides.sum()
    else:
        beta_norm = beta_peptides
    
    # Predict on spectral part
    y_pred = X_base @ beta_peptides
    cos = cosine(y_base, y_pred)
    
    # Per-peptide results
    results = []
    for j, psm in enumerate(sd['psm_info']):
        results.append({
            'spectrum_key': sd['spectrum_key'],
            'peptide': psm['peptide'],
            'model': model_name,
            'lambda': lambda_val,
            'beta': float(beta_norm[j]),
            'by_int_frac': psm.get('by_int_frac', np.nan),
            'prosit_cosine': psm.get('prosit_cosine', np.nan),
            'hyperscore': psm['hyperscore'],
            'n_psm': sd['n_psm'],
            'cosine': cos
        })
    
    return results

print(f"\n{'='*70}")
print("VALIDATION WITH OPTIMAL LAMBDA")
print(f"{'='*70}")

val_results = []

# Model 1: LASSO (2+ PSM)
model_name = 'LASSO (2+ PSM)'
lam_opt = optimal[model_name]
lam_str = "NNLS" if lam_opt == 0 else f"{lam_opt:.0e}"
print(f"\nValidating {model_name} (λ={lam_str})...")

tasks = [(sd, False, lam_opt, model_name) for sd in val_2plus]

t0 = time.time()
with Pool(cpu_count()) as pool:
    results = pool.map(validate_full, tasks, chunksize=100)
t1 = time.time()

for batch in results:
    val_results.extend(batch)

print(f"  Completed in {t1-t0:.1f}s")

# Model 2: LASSO (3+ PSM)
model_name = 'LASSO (3+ PSM)'
lam_opt = optimal[model_name]
lam_str = "NNLS" if lam_opt == 0 else f"{lam_opt:.0e}"
print(f"\nValidating {model_name} (λ={lam_str})...")

tasks = [(sd, False, lam_opt, model_name) for sd in val_3plus]

t0 = time.time()
with Pool(cpu_count()) as pool:
    results = pool.map(validate_full, tasks, chunksize=100)
t1 = time.time()

for batch in results:
    val_results.extend(batch)

print(f"  Completed in {t1-t0:.1f}s")

# Model 3: LASSO+MS1 (3+ PSM)
model_name = 'LASSO+MS1 (3+ PSM)'
lam_opt = optimal[model_name]
lam_str = "NNLS" if lam_opt == 0 else f"{lam_opt:.0e}"
print(f"\nValidating {model_name} (λ={lam_str})...")

tasks = [(sd, True, lam_opt, model_name) for sd in val_3plus_ms1]

t0 = time.time()
with Pool(cpu_count()) as pool:
    results = pool.map(validate_full, tasks, chunksize=100)
t1 = time.time()

for batch in results:
    val_results.extend(batch)

print(f"  Completed in {t1-t0:.1f}s")

# Save results
df_val = pd.DataFrame(val_results)
df_val.to_csv('/tmp/lasso_validation_three_models.csv', index=False)

print(f"\n✓ Saved: /tmp/lasso_validation_three_models.csv")
print(f"  Total PSMs: {len(df_val):,}")
print(f"  Total spectra: {df_val['spectrum_key'].nunique():,}")
print(f"\n  Results per model:")
for model in df_val['model'].unique():
    n_spectra = df_val[df_val['model'] == model]['spectrum_key'].nunique()
    n_psms = len(df_val[df_val['model'] == model])
    print(f"    {model:25s}: {n_spectra:,} spectra, {n_psms:,} PSMs")


VALIDATION WITH OPTIMAL LAMBDA

Validating LASSO (2+ PSM) (λ=NNLS)...
  Completed in 8.0s

Validating LASSO (3+ PSM) (λ=NNLS)...
  Completed in 8.0s

Validating LASSO+MS1 (3+ PSM) (λ=1e-04)...
  Completed in 8.5s

✓ Saved: /tmp/lasso_validation_three_models.csv
  Total PSMs: 220,717
  Total spectra: 51,548

  Results per model:
    LASSO (2+ PSM)           : 29,991 spectra, 92,645 PSMs
    LASSO (3+ PSM)           : 20,044 spectra, 72,802 PSMs
    LASSO+MS1 (3+ PSM)       : 15,357 spectra, 55,270 PSMs


In [20]:
# ============================================================
# VISUALIZATION: BETA PREDICTIONS vs GROUND TRUTH
# ============================================================

print(f"\n{'='*70}")
print("BETA PREDICTION QUALITY")
print(f"{'='*70}")

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

COLOR_2PLUS = '#E63946'
COLOR_3PLUS = '#F77F00'
COLOR_MS1 = '#457B9D'

colors = {
    'LASSO (2+ PSM)': COLOR_2PLUS,
    'LASSO (3+ PSM)': COLOR_3PLUS,
    'LASSO+MS1 (3+ PSM)': COLOR_MS1
}

# ============================================================
# ROW 1: BETA vs FRAGSHARE (by_int_frac)
# ============================================================

for idx, model in enumerate(['LASSO (2+ PSM)', 'LASSO (3+ PSM)', 'LASSO+MS1 (3+ PSM)']):
    ax = axes[0, idx]
    
    subset = df_val[df_val['model'] == model]
    valid = subset.dropna(subset=['by_int_frac'])
    valid = valid[valid['by_int_frac'] > 0]
    
    if len(valid) > 0:
        # Sample for visualization
        sample = valid.sample(min(3000, len(valid)), random_state=42)
        
        # Scatter plot
        ax.scatter(sample['by_int_frac'], sample['beta'], 
                  alpha=0.3, s=10, color=colors[model])
        
        # Perfect correlation line
        ax.plot([0, 1], [0, 1], 'k--', linewidth=2, alpha=0.7, label='Perfect')
        
        # Compute correlation
        corr = valid['beta'].corr(valid['by_int_frac'])
        
        # Add text box with correlation
        ax.text(0.05, 0.95, f'r = {corr:.3f}\nn = {len(valid):,}',
                transform=ax.transAxes, fontsize=11, fontweight='bold',
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        ax.set_xlabel('FragShare (Ground Truth)', fontweight='bold', fontsize=11)
        ax.set_ylabel('β (Predicted)', fontweight='bold', fontsize=11)
        ax.set_title(f'{model}\nβ vs FragShare', fontweight='bold', fontsize=12)
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.grid(alpha=0.3)
        ax.legend(fontsize=9)

# ============================================================
# ROW 2: BETA vs PROSIT COSINE
# ============================================================

for idx, model in enumerate(['LASSO (2+ PSM)', 'LASSO (3+ PSM)', 'LASSO+MS1 (3+ PSM)']):
    ax = axes[1, idx]
    
    subset = df_val[df_val['model'] == model]
    valid = subset.dropna(subset=['prosit_cosine'])
    
    if len(valid) > 0:
        # Sample for visualization
        sample = valid.sample(min(3000, len(valid)), random_state=42)
        
        # Scatter plot
        ax.scatter(sample['prosit_cosine'], sample['beta'], 
                  alpha=0.3, s=10, color=colors[model])
        
        # Compute correlation
        corr = valid['beta'].corr(valid['prosit_cosine'])
        
        # Add text box with correlation
        ax.text(0.05, 0.95, f'r = {corr:.3f}\nn = {len(valid):,}',
                transform=ax.transAxes, fontsize=11, fontweight='bold',
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        ax.set_xlabel('Prosit Cosine (Prediction)', fontweight='bold', fontsize=11)
        ax.set_ylabel('β (Predicted)', fontweight='bold', fontsize=11)
        ax.set_title(f'{model}\nβ vs Prosit Cosine', fontweight='bold', fontsize=12)
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(FIG_DIR / 'beta_predictions_quality.png', dpi=150, facecolor='white')
plt.show()

print(f"\n✓ Saved: {FIG_DIR / 'beta_predictions_quality.png'}")

# ============================================================
# SUMMARY TABLE: CORRELATIONS
# ============================================================

print(f"\n{'='*70}")
print("BETA PREDICTION CORRELATIONS - SUMMARY")
print(f"{'='*70}")

print(f"\n{'Model':25s} {'β~FragShare':>12} {'β~Prosit':>12} {'n_FragShare':>12} {'n_Prosit':>12}")
print("-" * 85)

for model in ['LASSO (2+ PSM)', 'LASSO (3+ PSM)', 'LASSO+MS1 (3+ PSM)']:
    subset = df_val[df_val['model'] == model]
    
    # FragShare correlation
    valid_frag = subset.dropna(subset=['by_int_frac'])
    valid_frag = valid_frag[valid_frag['by_int_frac'] > 0]
    corr_frag = valid_frag['beta'].corr(valid_frag['by_int_frac']) if len(valid_frag) > 0 else np.nan
    n_frag = len(valid_frag)
    
    # Prosit correlation
    valid_prosit = subset.dropna(subset=['prosit_cosine'])
    corr_prosit = valid_prosit['beta'].corr(valid_prosit['prosit_cosine']) if len(valid_prosit) > 0 else np.nan
    n_prosit = len(valid_prosit)
    
    print(f"{model:25s} {corr_frag:>12.3f} {corr_prosit:>12.3f} {n_frag:>12,} {n_prosit:>12,}")

print(f"\n{'='*70}")


BETA PREDICTION QUALITY


<Figure size 2160x1440 with 6 Axes>


✓ Saved: /tmp/lasso_figures/beta_predictions_quality.png

BETA PREDICTION CORRELATIONS - SUMMARY

Model                      β~FragShare     β~Prosit  n_FragShare     n_Prosit
-------------------------------------------------------------------------------------
LASSO (2+ PSM)                   0.661        0.628       92,639       92,645
LASSO (3+ PSM)                   0.703        0.663       72,801       72,802
LASSO+MS1 (3+ PSM)               0.570        0.621       55,270       55,270



In [19]:
# ============================================================
# PERFORMANCE ANALYSIS: THREE MODELS
# ============================================================

from scipy.stats import ttest_rel

print(f"\n{'='*70}")
print("PERFORMANCE ANALYSIS")
print(f"{'='*70}")

# Reconstruction quality
print(f"\n--- Reconstruction Quality ---")
print(f"\n{'Model':25s} {'Mean':>8s} {'Median':>8s} {'Std':>8s} {'n':>8s}")
print("-" * 70)

for model in ['LASSO (2+ PSM)', 'LASSO (3+ PSM)', 'LASSO+MS1 (3+ PSM)']:
    subset = df_val[df_val['model'] == model]
    cos_per_spectrum = subset.groupby('spectrum_key')['cosine'].first()
    
    print(f"{model:25s} {cos_per_spectrum.mean():>8.4f} {cos_per_spectrum.median():>8.4f} "
          f"{cos_per_spectrum.std():>8.4f} {len(cos_per_spectrum):>8,}")

# Beta correlations
print(f"\n{'='*70}")
print("BETA CORRELATIONS")
print(f"{'='*70}")

print(f"\n{'Model':25s} {'β~FragShare':>12} {'β~Prosit':>12} {'β~Hyper':>12}")
print("-" * 70)

for model in ['LASSO (2+ PSM)', 'LASSO (3+ PSM)', 'LASSO+MS1 (3+ PSM)']:
    subset = df_val[df_val['model'] == model]
    
    # Filter valid
    valid = subset.dropna(subset=['by_int_frac'])
    valid = valid[valid['by_int_frac'] > 0]
    
    corr_frag = valid['beta'].corr(valid['by_int_frac'])
    corr_prosit = valid['beta'].corr(valid['prosit_cosine'])
    corr_hyper = valid['beta'].corr(valid['hyperscore'])
    
    print(f"{model:25s} {corr_frag:>12.3f} {corr_prosit:>12.3f} {corr_hyper:>12.3f}")

# Statistical tests
print(f"\n{'='*70}")
print("STATISTICAL TESTS (Paired t-tests)")
print(f"{'='*70}")

# Test 1: LASSO (3+ PSM) vs LASSO (2+ PSM)
print(f"\n--- Test 1: LASSO (3+ PSM) vs LASSO (2+ PSM) ---")

cos_2plus = df_val[df_val['model'] == 'LASSO (2+ PSM)'].groupby('spectrum_key')['cosine'].first()
cos_3plus = df_val[df_val['model'] == 'LASSO (3+ PSM)'].groupby('spectrum_key')['cosine'].first()

# Find common spectra
common_spectra = sorted(set(cos_2plus.index) & set(cos_3plus.index))

if len(common_spectra) > 0:
    cos_2_aligned = cos_2plus.loc[common_spectra]
    cos_3_aligned = cos_3plus.loc[common_spectra]
    
    delta = cos_3_aligned.mean() - cos_2_aligned.mean()
    t_stat, p_val = ttest_rel(cos_3_aligned, cos_2_aligned)
    
    sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else "ns"
    
    print(f"  Common spectra: {len(common_spectra):,}")
    print(f"  Δ Cosine:       {delta:+.4f}")
    print(f"  t-statistic:    {t_stat:.3f}")
    print(f"  p-value:        {p_val:.3e}")
    print(f"  Significance:   {sig}")
    
    if delta > 0 and p_val < 0.05:
        print(f"  → LASSO (3+ PSM) is significantly BETTER")
    elif delta < 0 and p_val < 0.05:
        print(f"  → LASSO (2+ PSM) is significantly BETTER")
    else:
        print(f"  → No significant difference")
else:
    print(f"  No common spectra for comparison")

# Test 2: LASSO+MS1 (3+ PSM) vs LASSO (3+ PSM)
print(f"\n--- Test 2: LASSO+MS1 (3+ PSM) vs LASSO (3+ PSM) ---")

cos_lasso_3 = df_val[df_val['model'] == 'LASSO (3+ PSM)'].groupby('spectrum_key')['cosine'].first()
cos_ms1 = df_val[df_val['model'] == 'LASSO+MS1 (3+ PSM)'].groupby('spectrum_key')['cosine'].first()

# Find common spectra
common_spectra = sorted(set(cos_lasso_3.index) & set(cos_ms1.index))

if len(common_spectra) > 0:
    cos_lasso_aligned = cos_lasso_3.loc[common_spectra]
    cos_ms1_aligned = cos_ms1.loc[common_spectra]
    
    delta = cos_ms1_aligned.mean() - cos_lasso_aligned.mean()
    t_stat, p_val = ttest_rel(cos_ms1_aligned, cos_lasso_aligned)
    
    sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else "ns"
    
    print(f"  Common spectra: {len(common_spectra):,}")
    print(f"  Δ Cosine:       {delta:+.4f}")
    print(f"  t-statistic:    {t_stat:.3f}")
    print(f"  p-value:        {p_val:.3e}")
    print(f"  Significance:   {sig}")
    
    if delta > 0 and p_val < 0.05:
        print(f"  → LASSO+MS1 is significantly BETTER")
    elif delta < 0 and p_val < 0.05:
        print(f"  → LASSO (3+ PSM) is significantly BETTER")
    else:
        print(f"  → No significant difference")
else:
    print(f"  No common spectra for comparison")

# Performance by chimericity
print(f"\n{'='*70}")
print("PERFORMANCE BY CHIMERICITY")
print(f"{'='*70}")

for n_psm in sorted(df_val['n_psm'].unique()):
    print(f"\n{n_psm} PSM:")
    for model in ['LASSO (2+ PSM)', 'LASSO (3+ PSM)', 'LASSO+MS1 (3+ PSM)']:
        subset = df_val[(df_val['model'] == model) & (df_val['n_psm'] == n_psm)]
        if len(subset) > 0:
            cos_mean = subset.groupby('spectrum_key')['cosine'].first().mean()
            n_spec = subset['spectrum_key'].nunique()
            print(f"  {model:25s}: {cos_mean:.4f} (n={n_spec:,})")

print(f"\n{'='*70}")
print("✓ Performance analysis complete")
print(f"{'='*70}")


PERFORMANCE ANALYSIS

--- Reconstruction Quality ---

Model                         Mean   Median      Std        n
----------------------------------------------------------------------
LASSO (2+ PSM)              0.7945   0.8400   0.1646   29,991
LASSO (3+ PSM)              0.8026   0.8427   0.1516   20,044
LASSO+MS1 (3+ PSM)          0.7561   0.7839   0.1564   15,357

BETA CORRELATIONS

Model                      β~FragShare     β~Prosit      β~Hyper
----------------------------------------------------------------------
LASSO (2+ PSM)                   0.661        0.628        0.287
LASSO (3+ PSM)                   0.703        0.663        0.345
LASSO+MS1 (3+ PSM)               0.570        0.621        0.219

STATISTICAL TESTS (Paired t-tests)

--- Test 1: LASSO (3+ PSM) vs LASSO (2+ PSM) ---
  Common spectra: 6,075
  Δ Cosine:       +0.0000
  t-statistic:    nan
  p-value:        nan
  Significance:   ns
  → No significant difference

--- Test 2: LASSO+MS1 (3+ PSM) vs LASSO (3+

In [26]:
# ============================================================
# VISUALIZATION: MODEL COMPARISON
# ============================================================
import matplotlib.pyplot as plt
import seaborn as sns

print(f"\n{'='*70}")
print("CREATING VISUALIZATIONS")
print(f"{'='*70}")

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

COLOR_2PLUS = '#E63946'
COLOR_3PLUS = '#F77F00'
COLOR_MS1 = '#457B9D'

colors = {
    'LASSO (2+ PSM)': COLOR_2PLUS,
    'LASSO (3+ PSM)': COLOR_3PLUS,
    'LASSO+MS1 (3+ PSM)': COLOR_MS1
}

# Panel 1: Cosine Distribution
ax = axes[0, 0]

for model in ['LASSO (2+ PSM)', 'LASSO (3+ PSM)', 'LASSO+MS1 (3+ PSM)']:
    subset = df_val[df_val['model'] == model]
    cos_vals = subset.groupby('spectrum_key')['cosine'].first()
    
    ax.hist(cos_vals, bins=40, alpha=0.5, 
           label=model, color=colors[model], edgecolor='white')

ax.set_xlabel('Reconstruction Cosine', fontweight='bold', fontsize=11)
ax.set_ylabel('Count', fontweight='bold', fontsize=11)
ax.set_title('Reconstruction Quality Distribution', fontweight='bold', fontsize=12)
ax.legend(fontsize=9)
ax.grid(alpha=0.3, axis='y')

# Panel 2: Performance by Chimericity
ax = axes[0, 1]

for model in ['LASSO (2+ PSM)', 'LASSO (3+ PSM)', 'LASSO+MS1 (3+ PSM)']:
    means = []
    n_psms_list = []
    
    for n_psm in sorted(df_val['n_psm'].unique()):
        subset = df_val[(df_val['model'] == model) & (df_val['n_psm'] == n_psm)]
        if len(subset) > 0:
            mean_cos = subset.groupby('spectrum_key')['cosine'].first().mean()
            means.append(mean_cos)
            n_psms_list.append(n_psm)
    
    ax.plot(n_psms_list, means, marker='o', linewidth=2.5, 
           label=model, color=colors[model])

ax.set_xlabel('Number of PSMs', fontweight='bold', fontsize=11)
ax.set_ylabel('Mean Cosine', fontweight='bold', fontsize=11)
ax.set_title('Performance by Chimericity', fontweight='bold', fontsize=12)
ax.legend(fontsize=9)
ax.grid(alpha=0.3)

# Panel 3: Beta vs FragShare (3+ PSM models only)
ax = axes[1, 0]

for model in ['LASSO (3+ PSM)', 'LASSO+MS1 (3+ PSM)']:
    subset = df_val[df_val['model'] == model]
    valid = subset.dropna(subset=['by_int_frac'])
    valid = valid[valid['by_int_frac'] > 0]
    
    sample = valid.sample(min(2000, len(valid)), random_state=42)
    
    ax.scatter(sample['by_int_frac'], sample['beta'], 
              alpha=0.3, s=10, label=model, color=colors[model])

ax.plot([0, 1], [0, 1], 'k--', linewidth=1.5, alpha=0.5, label='Perfect correlation')
ax.set_xlabel('FragShare (Ground Truth)', fontweight='bold', fontsize=11)
ax.set_ylabel('β (Predicted)', fontweight='bold', fontsize=11)
ax.set_title('Beta vs FragShare (3+ PSM Models)', fontweight='bold', fontsize=12)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.legend(fontsize=9)
ax.grid(alpha=0.3)

# Panel 4: Boxplot Comparison
ax = axes[1, 1]

data_for_box = []
labels = []

for model in ['LASSO (2+ PSM)', 'LASSO (3+ PSM)', 'LASSO+MS1 (3+ PSM)']:
    subset = df_val[df_val['model'] == model]
    cos_vals = subset.groupby('spectrum_key')['cosine'].first()
    data_for_box.append(cos_vals.values)
    labels.append(model)

bp = ax.boxplot(data_for_box, labels=labels, patch_artist=True,
                showmeans=True, meanline=True)

for patch, model in zip(bp['boxes'], labels):
    patch.set_facecolor(colors[model])
    patch.set_alpha(0.6)

ax.set_ylabel('Reconstruction Cosine', fontweight='bold', fontsize=11)
ax.set_title('Performance Distribution by Model', fontweight='bold', fontsize=12)
ax.tick_params(axis='x', rotation=15)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIG_DIR / 'model_comparison_validation.png', dpi=150, facecolor='white')
plt.show()

print(f"\n✓ Saved: {FIG_DIR / 'model_comparison_validation.png'}")


CREATING VISUALIZATIONS


<Figure size 1680x1200 with 4 Axes>


✓ Saved: /tmp/lasso_figures/model_comparison_validation.png
